### Netflix EDA
- 아래의 서비스에서 데이터를 수집하여 EDA를 수행하세요.


- flixpatrol 사이트에서 데이터 수집
    - `https://flixpatrol.com/top10/netflix/world/2021/full/#netflix-1`


- kaggle에서 netflix 컨텐츠 데이터 수집
    - `https://www.kaggle.com/shivamb/netflix-shows`


- 컨텐츠의 등급데이터는 아래의 코드 사용
```
ratings_ages = {'TV-PG': 'Older Kids', 'TV-MA': 'Adults', 'TV-Y7-FV': 'Older Kids',
                  'TV-Y7': 'Older Kids', 'TV-14': 'Teens', 'R': 'Adults', 'TV-Y': 'Kids',
                  'NR': 'Adults', 'PG-13': 'Teens', 'TV-G': 'Kids', 'PG': 'Older Kids',
                  'G': 'Kids', 'UR': 'Adults', 'NC-17': 'Adults'}
```

#### EDA의 수행 절차
- 데이터 수집
- 결측 데이터의 처리
- 데이터 탐색
    - 수집한 데이터에서 자유롭게 주제를 선정하여 데이터 분석을 통한 인사이트 도출
    - 예시 : 인도영화는 다른 국가보다 러닝타임이 길것이다. 컨텐츠의 퀄리티가 가장 좋은 국가는?

### 한국 Top10 영화 데이터 수집

In [67]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [68]:
# 1. 웹서비스 분석 : URL

In [69]:
country = "south-korea"
year = "2021"
url = f"https://flixpatrol.com/top10/netflix/{country}/{year}/"
url

'https://flixpatrol.com/top10/netflix/south-korea/2021/'

In [70]:
# 2. request(url) > response(data) : data(html) *이때, html문자열은 한페이지 모두

In [71]:
response = requests.get(url)
response

<Response [200]>

In [72]:
response.text[:500]

'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n\t<title>TOP 10 on Netflix in South Korea in 2021 • FlixPatrol</title>\n\t<link rel="preload" as="font" type="font/woff2" crossorigin href="/static/fonts/Inter-roman.var.woff2?v=3.18" nonce="ObITytRtXq/jyzs7JIzFEQ==">\n\t<link rel="stylesheet" type="text/css" href="/static/dist/all.min.css?v=ec3a8b3a" nonce'

In [73]:
# 3. data(html) > bs_obj.select(css-selector) > text 

In [74]:
dom = BeautifulSoup(response.text, "html.parser")

In [75]:
elements = dom.select("#netflix-1 > div.-mx-content > div > div > table > tbody > tr")
len(elements)

10

In [76]:
# 각 데이터에서 필요한 정보 수집
element = elements[0]
data = {
    #strip() : 앞뒤 공백 문자 제거
    "no":element.select("td")[0].text.strip(),
    "title":element.select("td")[1].select_one("a").text.strip(),
    "points":element.select("td")[2].text.strip(),

}

data

{'no': '1.', 'title': '365 Days', 'points': '573'}

In [77]:
# [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
data = []
for element in elements:
    data.append({
        "no":element.select("td")[0].text.strip(),
        "title":element.select("td")[1].select_one("a").text.strip(),
        "points":element.select("td")[2].text.strip(),
    })
df = pd.DataFrame(data)
df.tail(2)

,no,title,points
8,9.,Parasite,241
9,10.,Samjin Company English Class,238


In [78]:
# function 함수화 : 국가별 Top10 영화 데이터 수집

In [79]:
def topmovies(country, year):
    # 1. 웹서비스 분석 : URL
    url = f"https://flixpatrol.com/top10/netflix/{country}/{year}/"
    # 2. request(url) > response(data) : data(html) *이때, html문자열은 한페이지 모두
    response = requests.get(url)
    # 3. data(html) > bs_obj.select(css-selector) > text 
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select("#netflix-1 > div.-mx-content > div > div > table > tbody > tr")
    element = elements[0]
    # [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
    data = []
    for element in elements:
        data.append({
            "no":element.select("td")[0].text.strip(),
            "title":element.select("td")[1].select_one("a").text.strip(),
            "points":element.select("td")[2].text.strip(),
        })
    df = pd.DataFrame(data)
    return df

In [80]:
country = "united-states"
year = "2022"
topmovies(country, year)

,no,title,points
0,1.,Don't Look Up,265
1,2.,Despicable Me 2,173
2,3.,The Tinder Swindler,165
3,4.,Just Go with It,148
4,5.,The Royal Treatment,117
5,6.,Journey 2: The Mysterious Island,109
6,7.,Despicable Me,107
7,8.,Brazen,103
8,9.,Home Team,99
9,10.,The Longest Yard,97


### 한국 Top10 드라마 데이터 수집

In [81]:
# 1. 웹서비스 분석 : URL

In [82]:
url = "https://flixpatrol.com/top10/netflix/south-korea/2021/"

In [83]:
# 2. request(url) > response(data) : data(html) *이때, html문자열은 한페이지 모두

In [84]:
response = requests.get(url)
response

<Response [200]>

In [85]:
response.text[:500]

'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n\t<title>TOP 10 on Netflix in South Korea in 2021 • FlixPatrol</title>\n\t<link rel="preload" as="font" type="font/woff2" crossorigin href="/static/fonts/Inter-roman.var.woff2?v=3.18" nonce="9GKKEvlJUH+ZaatWTBKCXw==">\n\t<link rel="stylesheet" type="text/css" href="/static/dist/all.min.css?v=ec3a8b3a" nonce'

In [86]:
# 3. data(html) > bs_obj.select(css-selector) > text 

In [87]:
dom = BeautifulSoup(response.text, "html.parser")

In [88]:
elements = dom.select("#netflix-2 > div.-mx-content > div > div > table > tbody > tr")
len(elements)

10

In [89]:
# 각 데이터에서 필요한 정보 수집
element = elements[0]
data = {
    #strip() : 앞뒤 공백 문자 제거
    "no":element.select("td")[0].text.strip(),
    "title":element.select("td")[1].select_one("a").text.strip(),
    "points":element.select("td")[2].text.strip(),

}

data

{'no': '1.', 'title': 'Hospital Playlist', 'points': '1,215'}

In [90]:
# [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
data = []
for element in elements:
    data.append({
        "no":element.select("td")[0].text.strip(),
        "title":element.select("td")[1].select_one("a").text.strip(),
        "points":element.select("td")[2].text.strip(),
    })
df = pd.DataFrame(data)
df.tail(2)

,no,title,points
8,9.,Sisyphus: The Myth,542
9,10.,The King's Affection,540


In [91]:
def topshows(country, year):
    # 1. 웹서비스 분석 : URL
    url = f"https://flixpatrol.com/top10/netflix/{country}/{year}/"
    # 2. request(url) > response(data) : data(html) *이때, html문자열은 한페이지 모두
    response = requests.get(url)
    # 3. data(html) > bs_obj.select(css-selector) > text 
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select("#netflix-2 > div.-mx-content > div > div > table > tbody > tr")
    element = elements[0]
    # [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
    data = []
    for element in elements:
        data.append({
            "no":element.select("td")[0].text.strip(),
            "title":element.select("td")[1].select_one("a").text.strip(),
            "points":element.select("td")[2].text.strip(),
        })

    return pd.DataFrame(data)

In [92]:
country = "united-states"
year = "2022"
topshows(country, year)

,no,title,points
0,1.,Ozark,262
1,2.,Cobra Kai,230
2,3.,All of Us are Dead,180
3,4.,Stay Close,172
4,5.,The Witcher,171
5,6.,Archive 81,167
6,7.,Sweet Magnolias,152
7,8.,CoComelon,151
8,9.,Cheer,133
9,10.,The Woman in the House Across the Street from ...,128


### Total Movies on Netflix in 2021

In [132]:
def totalmovies(year):
    # 1. 웹서비스 분석 : URL
    url = f"https://flixpatrol.com/top10/netflix/world/{year}/full/"
    # 2. request(url) > response(data) : data(html) *이때, html문자열은 한페이지 모두
    response = requests.get(url)
    # 3. data(html) > bs_obj.select(css-selector) > text 
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select("#netflix-1 > div.-mx-content > div > div > table > tbody > tr")
    element = elements[0]
    # [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
    data = []
    for element in elements:
        data.append({
            "title" : element.select("td")[2].text.strip(),
            "point" : element.select("td")[3].text.strip(),
            "countries" : element.select("td")[5].text.strip(),
            "point/countries" : element.select("td")[6].text.strip(),
            "days" : element.select("td")[7].text.strip(),
            "point/days" : element.select("td")[8].text.strip(),
        })
    totalmovies = pd.DataFrame(data)
    return totalmovies

In [133]:
year = "2021"
totalmovies = totalmovies(year)


In [134]:
totalmovies

,title,point,countries,point/countries,days,point/days
0,Red Notice,"25,918",90,288,48,540
1,We Can Be Heroes,"25,311",82,309,214,118
2,Army of the Dead,"18,888",89,212,92,205
3,Wish Dragon,"16,953",82,207,88,193
4,Army of Thieves,"15,081",90,168,52,290
...,...,...,...,...,...,...
171,Scoob!,202,1,202,28,7
172,The Son of Bigfoot,192,7,27,8,24
173,Ava,186,4,47,29,6
174,Bombshell,181,2,91,14,13


### Total TV shows on Netflix in 2021

In [95]:
def totalshows(year):
    # 1. 웹서비스 분석 : URL
    url = f"https://flixpatrol.com/top10/netflix/world/{year}/full/"
    # 2. request(url) > response(data) : data(html) *이때, html문자열은 한페이지 모두
    response = requests.get(url)
    # 3. data(html) > bs_obj.select(css-selector) > text 
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select("#netflix-2 > div.-mx-content > div > div > table > tbody > tr")
    element = elements[0]
    # [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
    data = []
    for element in elements:
        data.append({
           "title" : element.select("td")[2].text.strip(),
            "point" : element.select("td")[3].text.strip(),
            "countries" : element.select("td")[5].text.strip(),
            "point/countries" : element.select("td")[6].text.strip(),
            "days" : element.select("td")[7].text.strip(),
            "point/days" : element.select("td")[8].text.strip(),
        })
    totalshows = pd.DataFrame(data)
    return totalshows

In [96]:
year = "2021"
totalshows(year)

,title,point,countries,point/countries,days,point/days
0,Squid Game,"57,980",90,644,104,558
1,Money Heist,"44,382",91,488,364,122
2,Lupin,"42,690",83,514,256,167
3,Sex/Life,"34,609",82,422,141,245
4,Bridgerton,"33,973",82,414,146,233
...,...,...,...,...,...,...
147,Fatma,"1,680",44,38,47,36
148,Sweet Home,"1,656",22,75,50,33
149,Outlander,"1,642",22,75,104,16
150,Beauty and the Baker,"1,638",18,91,28,58


### flixpatrol + kaggle merge

In [97]:
netflix = pd.read_csv("netflix_titles.csv")
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [98]:
new = pd.merge(totalmovies, netflix, on = "title")
new

,title,point,countries,point/countries,days,point/days,show_id,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,We Can Be Heroes,"25,311",82,309,214,118,s1495,Movie,Robert Rodriguez,"YaYa Gosselin, Pedro Pascal, Priyanka Chopra, ...",United States,"December 25, 2020",2020,PG,101 min,"Children & Family Movies, Comedies",When alien invaders capture Earth’s superheroe...
1,Army of the Dead,"18,888",89,212,92,205,s854,Movie,Zack Snyder,"Dave Bautista, Ella Purnell, Omari Hardwick, G...",United States,"May 21, 2021",2021,R,148 min,"Action & Adventure, Horror Movies","After a zombie outbreak in Las Vegas, a group ..."
2,Wish Dragon,"16,953",82,207,88,193,s740,Movie,Chris Appelhans,"Jimmy Wong, John Cho, Constance Wu, Will Yun L...","China, United States, Canada","June 11, 2021",2021,PG,102 min,"Children & Family Movies, Comedies",Determined teen Din is longing to reconnect wi...
3,The Mitchells vs. The Machines,"14,902",82,182,74,201,s962,Movie,"Mike Rianda, Jeff Rowe","Danny McBride, Abbi Jacobson, Maya Rudolph, Mi...",NaN,"April 30, 2021",2021,PG,114 min,"Children & Family Movies, Comedies",A robot apocalypse put the brakes on their cro...
4,Fatherhood,"14,660",82,179,54,271,s686,Movie,Paul Weitz,"Kevin Hart, Alfre Woodard, Lil Rel Howery, DeW...",United States,"June 18, 2021",2021,PG-13,111 min,Dramas,"A widowed new dad copes with doubts, fears, he..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2012,"1,886",31,61,51,37,s1114,Movie,Roland Emmerich,"John Cusack, Amanda Peet, Chiwetel Ejiofor, Th...",United States,"April 1, 2021",2009,PG-13,158 min,"Action & Adventure, Sci-Fi & Fantasy",When a flood of natural disasters begins to de...
85,Wanted,"1,878",32,59,20,94,s633,Movie,Nibal Arakji,"Daad Rizk, Georges Diab, Sihame Haddad, George...",NaN,"June 28, 2021",2019,TV-14,90 min,"Comedies, International Movies",Four seniors embark on misadventures after bre...
86,Just Say Yes,"1,878",43,44,16,117,s1110,Movie,"Appie Boudellah, Aram van de Rest","Yolanthe Cabau, Noortje Herlaar, Kim-Lian van ...",Netherlands,"April 2, 2021",2021,TV-MA,98 min,"Comedies, International Movies, Romantic Movies",Incurable romantic Lotte finds her life upende...
87,Ava,186,4,47,29,6,s1585,Movie,Tate Taylor,"Jessica Chastain, Colin Farrell, John Malkovic...",United States,"December 7, 2020",2020,R,97 min,"Action & Adventure, Dramas",An elite assassin wrestling with doubts about ...


In [99]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 88
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            89 non-null     object
 1   point            89 non-null     object
 2   countries        89 non-null     object
 3   point/countries  89 non-null     object
 4   days             89 non-null     object
 5   point/days       89 non-null     object
 6   show_id          89 non-null     object
 7   type             89 non-null     object
 8   director         88 non-null     object
 9   cast             88 non-null     object
 10  country          65 non-null     object
 11  date_added       89 non-null     object
 12  release_year     89 non-null     int64 
 13  rating           89 non-null     object
 14  duration         89 non-null     object
 15  listed_in        89 non-null     object
 16  description      89 non-null     object
dtypes: int64(1), object(16)
memory usage:

In [100]:
netflix.loc[netflix["type"] == "TV Show"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2676 entries, 1 to 8803
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       2676 non-null   object
 1   type          2676 non-null   object
 2   title         2676 non-null   object
 3   director      230 non-null    object
 4   cast          2326 non-null   object
 5   country       2285 non-null   object
 6   date_added    2666 non-null   object
 7   release_year  2676 non-null   int64 
 8   rating        2674 non-null   object
 9   duration      2676 non-null   object
 10  listed_in     2676 non-null   object
 11  description   2676 non-null   object
dtypes: int64(1), object(11)
memory usage: 271.8+ KB


In [101]:
netflix.loc[netflix["type"] == "Movie"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6131 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6131 non-null   object
 1   type          6131 non-null   object
 2   title         6131 non-null   object
 3   director      5943 non-null   object
 4   cast          5656 non-null   object
 5   country       5691 non-null   object
 6   date_added    6131 non-null   object
 7   release_year  6131 non-null   int64 
 8   rating        6129 non-null   object
 9   duration      6128 non-null   object
 10  listed_in     6131 non-null   object
 11  description   6131 non-null   object
dtypes: int64(1), object(11)
memory usage: 622.7+ KB


In [102]:
ratings_ages = {'TV-PG': 'Older Kids', 'TV-MA': 'Adults', 'TV-Y7-FV': 'Older Kids',
                'TV-Y7': 'Older Kids', 'TV-14': 'Teens', 'R': 'Adults', 'TV-Y': 'Kids',
                'NR': 'Adults', 'PG-13': 'Teens', 'TV-G': 'Kids', 'PG': 'Older Kids',
                'G': 'Kids', 'UR': 'Adults', 'NC-17': 'Adults'}

for i in range(len(netflix["rating"])) :
    if netflix["rating"][i] in ratings_ages :
        netflix["rating"][i] = ratings_ages[netflix["rating"][i]]

<ipython-input-102-8f6ab5ce2ed1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix["rating"][i] = ratings_ages[netflix["rating"][i]]


### Genre of Top Movies on 2021 

In [37]:
# elements = dom.select("#netflix-1 > div.-mx-content > div > div > table > tbody > tr")

In [127]:
url = "https://flixpatrol.com/top10/netflix/world/2021/full/"

In [128]:
response = requests.get(url)
response

<Response [200]>

In [129]:
dom = BeautifulSoup(response.text, "html.parser")

In [130]:
elements = dom.select("#netflix-1 > div.-mx-content > div > div > table > tbody > tr")
len(elements)

176

In [131]:
for element in elements:
    
    print("https://flixpatrol.com/" + element.select("td")[2].select_one("a").get("href"))

https://flixpatrol.com//title/red-notice/
https://flixpatrol.com//title/we-can-be-heroes-2020/
https://flixpatrol.com//title/army-of-the-dead/
https://flixpatrol.com//title/wish-dragon/
https://flixpatrol.com//title/army-of-thieves/
https://flixpatrol.com//title/the-mitchells-vs-the-machines/
https://flixpatrol.com//title/fatherhood/
https://flixpatrol.com//title/the-unforgivable/
https://flixpatrol.com//title/the-guilty-2021/
https://flixpatrol.com//title/yes-day/
https://flixpatrol.com//title/love-hard/
https://flixpatrol.com//title/outside-the-wire/
https://flixpatrol.com//title/love-and-monsters/
https://flixpatrol.com//title/kate/
https://flixpatrol.com//title/vivo/
https://flixpatrol.com//title/minions/
https://flixpatrol.com//title/back-to-the-outback/
https://flixpatrol.com//title/hes-all-that/
https://flixpatrol.com//title/sweet-girl/
https://flixpatrol.com//title/blood-red-sky/
https://flixpatrol.com//title/major-grom-plague-doctor/
https://flixpatrol.com//title/my-little-pon

In [135]:
for element in elements:
    url = "https://flixpatrol.com/" + element.select("td")[2].select_one("a").get("href")
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    country = dom.select_one("body > div.content.mt-4 > div > div.flex-grow > div.mb-6 > div.flex.flex-wrap.text-gray-500 > div > span:nth-child(3)").text

    totalmovies["country"] = country

In [123]:
for element in elements:
    url = "https://flixpatrol.com/" + element.select("td")[2].select_one("a").get("href")
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    try:
        gerne = dom.select_one("body > div.content.mt-4 > div > div.flex-grow > div.mb-6 > div.flex.flex-wrap.text-gray-500 > div > span:nth-child(9)").text
        totalmovies["genre"] = genre
    except:
        totalmovies["genre"] = "no genre"

In [121]:
totalmovies["gerne"].value_counts()

Heist    176
Name: gerne, dtype: int64

In [122]:
totalmovies.head(10)

,title,point,countries,point/countries,days,point/days,country,gerne
0,Red Notice,"25,918",90,288,48,540,United States,Heist
1,We Can Be Heroes,"25,311",82,309,214,118,United States,Heist
2,Army of the Dead,"18,888",89,212,92,205,United States,Heist
3,Wish Dragon,"16,953",82,207,88,193,United States,Heist
4,Army of Thieves,"15,081",90,168,52,290,United States,Heist
5,The Mitchells vs. The Machines,"14,902",82,182,74,201,United States,Heist
6,Fatherhood,"14,660",82,179,54,271,United States,Heist
7,The Unforgivable,"13,458",89,151,20,673,United States,Heist
8,The Guilty,"12,930",83,156,43,301,United States,Heist
9,Yes Day,"12,809",81,158,66,194,United States,Heist


In [ ]:
def nation(url):

    response = requests.get(url)

    dom = BeautifulSoup(response.text, "html.parser")

    country = dom.select_one("body > div.content.mt-4 > div > div.flex-grow > div.mb-6 > div.flex.flex-wrap.text-gray-500 > div > span:nth-child(3)").text

    return country



def genre(url):

    response = requests.get(url)

    dom = BeautifulSoup(response.text, "html.parser")

    fail = []

    genre = dom.select_one("body > div.content.mt-4 > div > div.flex-grow > div.mb-6 > div.flex.flex-wrap.text-gray-500 > div > span:nth-child(9)")

    return genre

In [ ]:
nations = []
for i in movies_df["link"] :
    nations.append(nation(i))

genres = []
for i in movies_df["link"] :
    genres.append(genre(i))

### 